# Task

Keep using the One-piece [competition case](https://www.kaggle.com/t/f5f7783abf31495f9593b3d93a18f9eb).

This time, it is based on the multiple linear regression framework as in the Assignment 2 (yes, as a TV series).
$$y=x'\beta + \epsilon.$$


1. Have you transformed any variables?
Polynomial transformation? Interactions?
1. Week 6 will talk about LOWESS, a simple nonparametric method.
1. Model averaging (or model assembly in ML language, stacking in particular). If you have a few competitive models, sometimes, the average of their predictions is better than any single model's prediction.
1. Get the test sample for prediction and submit your results on Kaggle to get your Kaggle score screenshot. Show the screenshot in the PDF file.

**Note:**
- All instructions in Assignment 1 apply.
- Use $\leq 300$ words.